# Image Classification using `sklearn.svm`

In [4]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
import skimage
from skimage.io import imread
from skimage.transform import resize

### Load images in structured directory like it's sklearn sample dataset

In [5]:
def load_image_files(container_path, dimension=(64, 64)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = skimage.io.imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [6]:
image_dataset = load_image_files("images/")

### Split data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

### Train data with parameter optimization

In [8]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}])

### Predict

In [14]:
y_pred = clf.predict(X_test)
print(y_pred)

[2 3 3 2 3 3 3 0 3 3 0 3 3 3 2 3 3 0 1 4 3 3 3 3 3 3 3 3 2 1 3 3 3 3 3 0 3
 4 3 4 3 2 3 1 0 3 4 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 4 3 3 3 1 3 3 0
 4 3 3 3 3 3 3 3 2 3 3 3 2 1 3 3 3 3 4 3 3 3 3 0 3 2 4 3 3 1 3 4 3 3 3 3 3
 4 2 1 2 3 3 3 2 3 3 3 3 3 3 3 3 2 0 3 3 3 1 3 3 3 0 3 3 3 4 0 3 3 3 3 3 3
 0 1 3 3 1 3 2 0 0 3 4 2 4 3 3 4 3 0 0 3 3 2 3 0 4 2 3 3 0 3 3 3 1 3 3 2 3
 3 1 3 3 3 3 2 3 4 3 0 3 4 4 1 3 3 4 3 3 3 3 3 3 3 1 0 3 4 2 3 3 4 3 3 4 3
 2 4 1 4 1 1 1 0 3 3 3 3 3 4 2 3 3 3 3 3 3 0 2 3 3 3 4 1 3 3 4 3 0 2 3 3 3
 2 0 2 0 3 3 3 3 0 3 3 3 3 0 4 3 3 3 3 3 2 1 3 3 3 3 4 2 0 3 2 4 2 0 3 3 1
 3 1 0 3 3 3 2 3 3 3 3 1 3 3 3 2 3 3 3 3 3 2 1 3 0 3 3 4 3 3 2 3 3 3 4 4 3
 3 3 2 3 0 3 3 2 4 3 3 4 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 2 1 3 1 3 2 1
 0 3 3 3 4 3 0 3 0 3 3 3 3 3 4 0 3 3 3 3 2 3 3 2 3 3 2 3 3 0 4 2 3 3 3 3 3
 2 3 3 1 4 3 3 0 3 3 4 2 3 4 3 0 3 2 1 1 2 3 3 2 3 3 3 3 3 3 3 3 3 2 3 3 1
 3 3 3 3 3 3 3 4 3 3 4 3 2 2 4 0 1 2 3 2 3 3 0 3 0 0 3 1 4 0 4 3 3 3 3 3 3
 0 3 3 1 3 3 3 3 3 4 1 3 

### Report

In [11]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

Classification report for - 
GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}]):
              precision    recall  f1-score   support

           0       0.56      0.50      0.53        64
           1       0.49      0.30      0.37        73
           2       0.64      0.44      0.53        81
           3       0.78      0.95      0.86       311
           4       0.69      0.58      0.63        71

    accuracy                           0.71       600
   macro avg       0.63      0.56      0.58       600
weighted avg       0.69      0.71      0.69       600




In [13]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.7133333333333334
